![Logo](https://www.up.edu.mx/wp-content/themes/panamericana/img/logo-footer.png)

# **Examen Final: Modelado score crediticio**

## **Asignatura:** Aprendizaje para grandes datos  
## **Profesor:** Raúl Martínez Noriega

---

**Nombre del Alumno:**  
Luis Enrique Noguera Gil

**Fecha de entrega:**  
09 de diciembre de 2024


# Auto reload modules

In [2]:
%load_ext autoreload
%autoreload 2

# Directories

In [3]:
# import findspark
# findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import fs
import os

In [4]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "../../spark-3.5.3-bin-hadoop3"

In [5]:
import findspark
findspark.init()

Preparación del ambiente de Spark

In [6]:
spark = (SparkSession.builder
  .master('local[*]')
  .appName('hello_world_spark')
  .getOrCreate())

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

24/12/09 19:34:00 WARN Utils: Your hostname, ThinkPad-X1-Nano resolves to a loopback address: 127.0.1.1; using 172.29.80.33 instead (on interface wlp0s20f3)
24/12/09 19:34:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/09 19:34:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


You are working with 1 core(s)


In [7]:
sc = spark.sparkContext

Mejorar la lectura de las tablas

In [8]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

Cargar archivos usando la librería fs para que sea agnóstico a cualquier ambiente distribuido o local.

In [21]:
RAW_DIR = fs.open_fs("../../data/raw")
CSV_DIR = RAW_DIR.getsyspath("score_de_credito.csv")

REPORTS_DIR = fs.open_fs("../../reports")

In [10]:
filePath = CSV_DIR

rawDF = spark.read.csv(filePath,
                       header="true",
                       inferSchema="true",
                       multiLine="true",
                       escape='"')

rawDF.show(3, truncate=False)

24/12/09 19:35:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+-----------+--------+-------------+----+-----------+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------------------------------------------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+---------------------+---------------------+-------------------+-----------------------+-------------------------------+------------------+------------+
|ID    |Customer_ID|Month   |Name         |Age |SSN        |Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Type_of_Loan                                                       |Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age   |Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|Payment_Behav

# EDA

# Hack Usando python

Generación de Profiling de python para ahorrar tiempo. Esto se asume que se está tomando una muestra de un set de datos muy grande. 

In [ ]:
df = rawDF.toPandas()
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NA,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [ ]:
from ydata_profiling import ProfileReport
Profile_DIR = REPORTS_DIR.getsyspath("EDA_REPORT_DIR")

#Generate Profile Report
report = ProfileReport(df, title= 'Report', html={'style':{'full_width':True}})
report.to_file(output_file=Profile_DIR)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

# Usando PySpark

1. ¿Cuántos registros hay?

In [12]:
rawDF.count()

100000

¿Habrá duplicados?

In [14]:
rawDF.dropDuplicates()
rawDF.count()

100000

No los hay.

¿Qué tipo de datos se tiene?

In [15]:
rawDF.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: string (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Num_of_Loan: string (nullable = true)
 |-- Type_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: string (nullable = true)
 |-- Changed_Credit_Limit: string (nullable = true)
 |-- Num_Credit_Inquiries: double (nullable = true)
 |-- Credit_Mix: string (nullable = true)
 |-- Outstanding_Debt: string (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Credit_History_Age: string (nullable = true

Vamos a seleccionar las variables que me interesan analizar, ya que por tiempo no se pueden revisar minuciosamente todas.

In [16]:
rawDF.columns

['ID',
 'Customer_ID',
 'Month',
 'Name',
 'Age',
 'SSN',
 'Occupation',
 'Annual_Income',
 'Monthly_Inhand_Salary',
 'Num_Bank_Accounts',
 'Num_Credit_Card',
 'Interest_Rate',
 'Num_of_Loan',
 'Type_of_Loan',
 'Delay_from_due_date',
 'Num_of_Delayed_Payment',
 'Changed_Credit_Limit',
 'Num_Credit_Inquiries',
 'Credit_Mix',
 'Outstanding_Debt',
 'Credit_Utilization_Ratio',
 'Credit_History_Age',
 'Payment_of_Min_Amount',
 'Total_EMI_per_month',
 'Amount_invested_monthly',
 'Payment_Behaviour',
 'Monthly_Balance',
 'Credit_Score']

¿Hay clientes repetidos?

In [19]:
rawDF.select("Customer_ID").distinct().count()

12500

In [20]:
rawDF.select("Customer_ID").count()

100000

Por lo tanto hay 12,500 clientes.

¿Hay datos faltantes? Pero aquí mejor dicho serán datos nulos previamente definidos ya que hay datos incorrectos en el set de datos, que no serán marcados como nulos.

Referencia https://stackoverflow.com/questions/44627386/how-to-find-count-of-null-and-nan-values-for-each-column-in-a-pyspark-dataframe

In [36]:
(
    rawDF
    .select([F.count(F.when(F.isnull(c), c)).alias(c) for c in rawDF.columns])
).show()

+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+------------------+---------------------+-------------------+-----------------------+-----------------+---------------+------------+
| ID|Customer_ID|Month|Name|Age|SSN|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|Payment_Behaviour|Monthly_Balance|Credit_Score|
+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+---

In [ ]:
(
    rawDF
    .summary()    
).show()

+-------+-------+-----------+------+------+------------------+-----------+----------+------------------+---------------------+------------------+------------------+------------------+-----------------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+------------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+--------------------+------------+
|summary|     ID|Customer_ID| Month|  Name|               Age|        SSN|Occupation|     Annual_Income|Monthly_Inhand_Salary| Num_Bank_Accounts|   Num_Credit_Card|     Interest_Rate|      Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|  Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|     Monthly_Balance|Cred

Primero hay que validar cúantos datos por cliente tenemos.

In [42]:
(
    rawDF
    .groupBy("Customer_ID")
    .count()
).show()

+-----------+-----+
|Customer_ID|count|
+-----------+-----+
| CUS_0x33d2|    8|
|  CUS_0xc65|    8|
| CUS_0x36ab|    8|
| CUS_0x7250|    8|
| CUS_0x9e13|    8|
|  CUS_0xf6c|    8|
| CUS_0x80b3|    8|
| CUS_0x42fb|    8|
| CUS_0x5479|    8|
| CUS_0x834f|    8|
| CUS_0x74f9|    8|
| CUS_0x684a|    8|
| CUS_0x63e0|    8|
| CUS_0x4e0d|    8|
| CUS_0x9f4c|    8|
| CUS_0xa980|    8|
| CUS_0x6371|    8|
| CUS_0x878c|    8|
| CUS_0xc064|    8|
| CUS_0x74c9|    8|
+-----------+-----+
only showing top 20 rows



¿Cuántos meses hay?

In [43]:
(
    rawDF
    .groupBy("Month")
    .count()
).show()

+--------+-----+
|   Month|count|
+--------+-----+
|    July|12500|
|February|12500|
| January|12500|
|   March|12500|
|     May|12500|
|  August|12500|
|   April|12500|
|    June|12500|
+--------+-----+



Hay 8 meses y en todos hay datos, pero no todos son correctos. Por tanto necesito identificar aquellos clientes que tienen toda la información correcta.

Vamos a eliminar variables que no tienen sentido. O bien, filtrar los datos que son plausibles de ser reales.

In [54]:
Float_cols = ["Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Num_Credit_Card", "Interest_Rate", "Num_of_Loan",
              "Delay_from_due_date", "Num_of_Delayed_Payment", "Changed_Credit_Limit", "Num_Credit_Inquiries",
              "Outstanding_Debt","Credit_Utilization_Ratio", "Total_EMI_per_month", "Amount_invested_monthly", 
              "Payment_Behaviour","Monthly_Balance"]

In [ ]:
String_cols = ["Month", "Occupation"]

Referencia https://stackoverflow.com/questions/53743795/how-to-check-if-a-string-column-in-pyspark-dataframe-is-all-numeric

In [101]:
rawDF_filtered = (
    rawDF
    .filter(
        # Tomando edades reales o plausibles o sin errores
        (F.col("Age").between(15, 80)),
    )
    # Tomando Valores que sean String
    .filter(
        (F.col("Month").cast("string").isNotNull())
    )
    .filter(
        (F.col("Occupation").cast("string").isNotNull())
    )
    # Se omite, puesto que quita muchos clientes dejarla lo reduce a 13
    # .filter(
    #     (F.col("Credit_Mix").cast("string").isNotNull()) & 
    #     (F.col("Credit_Mix") != "_")
    # )
    # Tomando Valores que sean flotantes
    .filter(
        (F.col("Annual_Income").cast("float").isNotNull())
    )
.   filter(
        (F.col("Annual_Income").cast("float").isNotNull())
    )
    .filter(
        (F.col("Monthly_Inhand_Salary").cast("float").isNotNull())
    )
    .filter(
        (F.col("Num_Bank_Accounts").cast("float").isNotNull())
    )
    .filter(
        (F.col("Num_Credit_Card").cast("float").isNotNull())
    )
    .filter(
        (F.col("Interest_Rate").cast("float").isNotNull())
    )
    .filter(
        (F.col("Num_of_Loan").cast("float").isNotNull())
    )
    .filter(
        (F.col("Delay_from_due_date").cast("float").isNotNull())
    )
    .filter(
        (F.col("Num_of_Delayed_Payment").cast("float").isNotNull())
    )
    .filter(
        (F.col("Changed_Credit_Limit").cast("float").isNotNull())
    )
    .filter(
        (F.col("Num_Credit_Inquiries").cast("float").isNotNull())
    )
    .filter(
        (F.col("Outstanding_Debt").cast("float").isNotNull())
    )
    .filter(
        (F.col("Credit_Utilization_Ratio").cast("float").isNotNull())
    )
    .filter(
        (F.col("Total_EMI_per_month").cast("float").isNotNull())
    )
    .filter(
        (F.col("Amount_invested_monthly").cast("float").isNotNull())
    )
    .filter(
        (F.col("Monthly_Balance").cast("float").isNotNull())
    )
)
rawDF_filtered.count()

53067

Seguro se puede optimizar pero es lo más sencillo que puedo hacer 

In [102]:
(
    rawDF_filtered
    .groupBy("Month", "Customer_ID") 
    .count()
    .groupBy("Month")
    .agg(F.countDistinct("Customer_ID").alias("Clientes_unicos"))
).show()

+--------+---------------+
|   Month|Clientes_unicos|
+--------+---------------+
|    July|           6738|
|February|           6513|
| January|           6626|
|   March|           6589|
|     May|           6629|
|  August|           6665|
|   April|           6654|
|    June|           6653|
+--------+---------------+



Nos vamos a quedar con los clientes que tienen todos los meses completos

In [103]:
customers_with_8_months = (
    rawDF_filtered
    .groupBy("Customer_ID")
    .agg(F.collect_set("Month").alias("Months"))
    .filter(F.size("Months") == 8)
)

filtered_df = rawDF_filtered.join(
    customers_with_8_months.select("Customer_ID"),
    on="Customer_ID",
    how="inner"
)

In [ ]:
filtered_df.show(truncate=False)

+-----------+------+--------+-------+---+-----------+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+------------------------------------------------------------------------------------------------------------------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+----------------------+---------------------+-------------------+-----------------------+--------------------------------+------------------+------------+
|Customer_ID|ID    |Month   |Name   |Age|SSN        |Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Type_of_Loan                                                                                                                  |Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_R

Validar datos

¿Cuántos registros por mes hay?

In [105]:
(
    filtered_df
    .groupBy("Month", "Customer_ID") 
    .count()
    .groupBy("Month")
    .agg(F.countDistinct("Customer_ID").alias("Clientes_unicos"))
).show()

+--------+---------------+
|   Month|Clientes_unicos|
+--------+---------------+
|    July|             73|
|February|             73|
| January|             73|
|   March|             73|
|     May|             73|
|  August|             73|
|   April|             73|
|    June|             73|
+--------+---------------+



Sólo me quedaré con las columnas que me hacen algo de sentido para procesarlos fácilmente. Para eso se requiere de negocio. Para el resto de las variables numéricas lo que haría es quitar aquellos que estén fuera del IQR por ejemplo, también checar la distribución etc. Pero por tiempo lo dejaré hasta aquí.

La idea es al menos garantizar que tengo el mismo cliente durante todo el tiempo de análisis.

Variables que no se procesaran pero son importantes:

`Type_of_Loan` Es importante pero como son varias las que están en un arreglo tardaría en procesarlo, así que no la consideraré.

`Credit_Mix` Se elimina por que hay muchos faltantes.

`Credit_History_Age` Se elimina por practicidad, pero se puede extraer buena información procesandola.

`Payment_Behaviour` Se dejara para pruebas.

Se podría considerar la nulidad del 80% pero no se considerará por tiempo.


In [ ]:
Data type string of column Annual_Income is not supported.
Data type string of column Num_of_Loan is not supported.
Data type string of column Num_of_Delayed_Payment is not supported.
Data type string of column Changed_Credit_Limit is not supported.
Data type string of column Outstanding_Debt is not supported.
Data type string of column Amount_invested_monthly is not supported.
Data type string of column Monthly_Balance is not supported.

In [178]:
df_clean = (
    filtered_df
    .drop("ID", "Name", "SSN", "Type_of_Loan", "Credit_Mix", "Credit_History_Age")
    .select(
        "*",
        (F.col("Annual_Income").cast("float")).alias("Annual_Income"),
        (F.col("Num_of_Loan").cast("float")).alias("Num_of_Loan"),
        (F.col("Num_of_Delayed_Payment").cast("float")).alias("Num_of_Delayed_Payment"),
        (F.col("Changed_Credit_Limit").cast("float")).alias("Changed_Credit_Limit"),
        (F.col("Outstanding_Debt").cast("float")).alias("Outstanding_Debt"),
        (F.col("Amount_invested_monthly").cast("float")).alias("Amount_invested_monthly"),
        (F.col("Monthly_Balance").cast("float")).alias("Monthly_Balance"),
        (F.col("Age").cast("float")).alias("Age")
    )
)
df_clean.show(5)

+-----------+--------+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------------+------------------------+---------------------+-------------------+-----------------------+--------------------+------------------+------------+-------------+-----------+----------------------+--------------------+----------------+-----------------------+---------------+----+
|Customer_ID|   Month|Age|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Outstanding_Debt|Credit_Utilization_Ratio|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|   Monthly_Balance|Credit_Score|Annual_Income|Num_of_Loan|Num_of_Delayed_Payment|Changed_Credit_Limit|Outstanding_Debt|Amount_invested_monthl

In [179]:
df_clean = filtered_df.drop("ID", "Name", "SSN", "Type_of_Loan", "Credit_Mix", "Credit_History_Age")

df_clean = df_clean.withColumn("Annual_Income", F.col("Annual_Income").cast("float"))
df_clean = df_clean.withColumn("Num_of_Loan", F.col("Num_of_Loan").cast("float"))
df_clean = df_clean.withColumn("Num_of_Delayed_Payment", F.col("Num_of_Delayed_Payment").cast("float"))
df_clean = df_clean.withColumn("Changed_Credit_Limit", F.col("Changed_Credit_Limit").cast("float"))
df_clean = df_clean.withColumn("Outstanding_Debt", F.col("Outstanding_Debt").cast("float"))
df_clean = df_clean.withColumn("Amount_invested_monthly", F.col("Amount_invested_monthly").cast("float"))
df_clean = df_clean.withColumn("Monthly_Balance", F.col("Monthly_Balance").cast("float"))
df_clean = df_clean.withColumn("Age", F.col("Age").cast("float"))

df_clean.show(5)


+-----------+--------+----+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------------+------------------------+---------------------+-------------------+-----------------------+--------------------+---------------+------------+
|Customer_ID|   Month| Age|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Outstanding_Debt|Credit_Utilization_Ratio|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|Credit_Score|
+-----------+--------+----+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------------+--------

In [180]:
df_clean.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: float (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Num_of_Loan: float (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: float (nullable = true)
 |-- Changed_Credit_Limit: float (nullable = true)
 |-- Num_Credit_Inquiries: double (nullable = true)
 |-- Outstanding_Debt: float (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Payment_of_Min_Amount: string (nullable = true)
 |-- Total_EMI_per_month: double (nullable = true)
 |-- Amount_invested_monthly: float (nullable = true)
 |-- Payment_Behaviour: string (nullable = true)
 |-- Monthly_Balance: float (nullable 

# Creación del pipeline del modelo.

In [181]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler

from pyspark.ml.classification import DecisionTreeClassifier

Probablemente al poner aleatorio todo, no me llevaría los cientes correctamente, pero para fines de prueba lo dejaré así.

Lo correcto es tomar el 80 % de clientes únicos y probar con el resto de los 20

In [182]:
df_train, df_test = df_clean.randomSplit([0.8,0.2], seed=43)


### Transformación de los datos

Cuando utilizamos árboles de decisión no debemos utilizar one-hot enconding para las variables categóricas. Sin embargo, si es necesario codificarlas como variables categóricas numéricas.

In [183]:
#--Si la columna es de tipo texto, la designamos como categórica
categoricalCols = [field for (field, dataType) in df_train.dtypes if dataType == "string"]

#--creamos los nombres de salida
indexOutputCols = [x + "Index" for x in categoricalCols]

#--definimos nuestro transformador
stringIndexer = StringIndexer(inputCols=categoricalCols,
                              outputCols=indexOutputCols,
                              handleInvalid="skip")

In [184]:
#--selecionamos las columnas numéricas excepto la variable respuesta
numericCols = [field for (field, dataType) in df_train.dtypes if ((dataType == "double") & (field != "price"))]

#--combinamos los nombres de variables categóricas (salida stringIndexer) y las numéricas
assemblerInputs = indexOutputCols + numericCols

#--se combinan todos los atributos en una sola columna
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

## Árbol de decisión

El primer algoritmo que vamos a entrenar, es un árbol de decisión

In [185]:
#--definimos nuestro estimador
dt_model = DecisionTreeClassifier(labelCol="Credit_Score")

In [186]:
#--definimos el estimador pipeline
pipeline = Pipeline(stages=[stringIndexer,
                            vecAssembler,
                            dt_model])

#--lo entrenamos
pipelineModel = pipeline.fit(df_train)

IllegalArgumentException: requirement failed: Column Credit_Score must be of type numeric but was actually of type string.

In [163]:

# Construct a pipeline
pipeline = Pipeline(stages=[indexer, onehot, assembler, regression])

# Train the pipeline on the training data
pipeline = pipeline.fit(flights_train)

# Make predictions on the testing data
predictions = pipeline.transform(flights_test)

NameError: name 'indexer' is not defined

Confusion matrix

In [ ]:
prediction.groupBy("label", "prediction").count().show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator()
evaluator.evaluate(prediction, {evaluator.metricName: 'weightedPrecision'})
evaluator.evaluate(prediction, {evaluator.metricName: 'accuracy'})

In [187]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [129]:
df_clean.show()

+-----------+--------+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------------+------------------------+---------------------+-------------------+-----------------------+--------------------+------------------+------------+
|Customer_ID|   Month|Age|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Outstanding_Debt|Credit_Utilization_Ratio|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|   Monthly_Balance|Credit_Score|
+-----------+--------+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------------+-----

In [164]:
df_clean.printSchema()

root
 |-- Customer_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: string (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: integer (nullable = true)
 |-- Num_Credit_Card: integer (nullable = true)
 |-- Interest_Rate: integer (nullable = true)
 |-- Num_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: integer (nullable = true)
 |-- Num_of_Delayed_Payment: string (nullable = true)
 |-- Changed_Credit_Limit: string (nullable = true)
 |-- Num_Credit_Inquiries: double (nullable = true)
 |-- Outstanding_Debt: string (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Payment_of_Min_Amount: string (nullable = true)
 |-- Total_EMI_per_month: double (nullable = true)
 |-- Amount_invested_monthly: string (nullable = true)
 |-- Payment_Behaviour: string (nullable = true)
 |-- Monthly_Balance: string (n

In [188]:
Cat_cols = ["Customer_ID", "Month", "Occupation", "Payment_of_Min_Amount", "Payment_Behaviour", "Credit_Score"]
indexOutputCols = [x + "Index" for x in Cat_cols]
Num_cols = ["Age", "Annual_Income", "Monthly_Inhand_Salary", "Num_Bank_Accounts", "Num_Credit_Card",
            "Interest_Rate", "Num_of_Loan", "Delay_from_due_date", "Num_of_Delayed_Payment",
            "Changed_Credit_Limit", "Num_Credit_Inquiries", "Outstanding_Debt", "Credit_Utilization_Ratio",
            "Total_EMI_per_month", "Amount_invested_monthly", "Monthly_Balance"]

In [152]:
len(Cat_cols)

6

In [151]:
indexOutputCols

['Customer_IDIndex',
 'MonthIndex',
 'OccupationIndex',
 'Payment_of_Min_AmountIndex',
 'Payment_BehaviourIndex',
 'Credit_ScoreIndex']

In [189]:
label_indexer = StringIndexer(inputCols=Cat_cols, outputCols=indexOutputCols)
data = label_indexer.fit(df_clean).transform(df_clean)

assembler = VectorAssembler(inputCols=Num_cols, outputCol="features")
data = assembler.transform(data)

In [194]:
data = data.withColumn("label", F.col("Credit_ScoreIndex"))


In [195]:
# Split data into training and testing sets
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

In [196]:
train_data.show()

+-----------+--------+----+------------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------+----------------------+--------------------+--------------------+----------------+------------------------+---------------------+-------------------+-----------------------+--------------------+---------------+------------+----------------+----------+---------------+--------------------------+----------------------+-----------------+--------------------+-----+
|Customer_ID|   Month| Age|  Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Outstanding_Debt|Credit_Utilization_Ratio|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|Monthly_Balance|Credit_Score|Customer_IDIndex|MonthIndex|OccupationIndex|Payment_of_Min_AmountIndex|Payment_BehaviourIndex|Credit_S

In [197]:
dt_classifier = DecisionTreeClassifier(labelCol="label", featuresCol="features")

model = dt_classifier.fit(train_data)

In [198]:
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(f"Test Accuracy: {accuracy:.2f}")

Test Accuracy: 0.74


# Conclusiones


Hay problemas en el casteo de las variables, por lo que el tipo de dato genera problemas. A final lo ideal es guardarlo en un parquet.

Para el tema del modelado, ya contar con un template listo para funcionar antes de tiempo para no tener que crearlo en caliente. 

Considerar tiempo en generar ese modelo base ya que se tiene todo el tema del preprocesamiento listo, pero se demoró mucho en generarlo. Ya que la aproximación probablemente sea la más adecuada la de analizar el cliente a lo largo del tiempo y evaluar otros diferentes. Pero el hacer esto demoró más tiempo de lo esperado.

Olvidé poner bien la etiqueta de label para la variable objetivo para que el modelo corriera.


Nuevo...


Analizar las metricas, hacer validación cruzada o optuna para encontrar los mejores parámetros...

Checar si funciona el modelo en la realidad, llevarlo a producción etc.

También la matriz de confusión y todas las métricas F1 scorre, etc.